In [ ]:
### author: Your name
### email: Your email   
### Date:

In [ ]:
!pip install d2l
!pip install matplotlib==3.0
!pip install matplotlib_inline

In [ ]:
from d2l import torch as d2l
import os
import torch     
from torch import nn  
##########################################################################################################################
# In this task, we choose a famous dataset, Large Movie Review Dataset (IMDB), as the Sentiment Analysis benchmark.
# The data link is shown in here: https://ai.stanford.edu/~amaas/data/sentiment/ .
##########################################################################################################################  

# **Assignment**
DIY your own sentiment analysis model by selecting different hypermeters.

1. File -> "save a copy in Drive" and open the copy file.
2. Randomly choose a batch size number.
3. Randomly choose a learning rate number.
4. Randomly choose a epoch number to train the model.
5. Randomly choose a dropout rate number.
6. Two lines of codeing: Increase the number of the sentiment_analysis_mlp layers from 2 to 3. (hint: one line coding on "__init__", while one line coding on "forward")
7. Run the model.
8. Generate the training acc, test acc visual graph.
9. Write a test case input sentence and return the answer (positive or negative).
10. Save this .ipynb and submit.


In [ ]:
##########################################################################################################################
# Set up the hyper-parameters.
print('-----------------------------start set up heper-parameter-----------------------------')
##########################################################################################################################
batch_size = ## randomly choose a number. e.g., 1, 2, 4, 8, 16, 32, 64, 128, ...

embed_size = ## fix 100
devices = d2l.try_all_gpus()

learning_rate =  ## randomly choose a number. e.g., 1e-1, 2e-1, 1e-2, 1e-3, ...
train_epochs = ## randomly choose a number. e.g., 1, 2, 3, 4, 5, ...

dropout_rate = ## randomly choose a number in the range from 0 to 1

sentence = ""   ## write a test case by yourself.

print('-----------------------------end set up heper-parameter----------------------------')

In [ ]:
##########################################################################################################################
# data preprocessing
print('-----------------------------start loading data and data preprocessing-----------------------------')
##########################################################################################################################
train_iter, test_iter, vocab = d2l.load_data_imdb(batch_size) ### data preprocessing will spend more time.
print('-----------------------------end loading data and data preprocessing-----------------------------')

In [ ]:
##########################################################################################################################
# Create the MLP model on Sentiment Analysis Task.
##########################################################################################################################
class Sentiment_Analysis_MLP(nn.Module):                                                     
    def __init__(self, vocab_size, embed_size, **kwargs):

        super(Sentiment_Analysis_MLP, self).__init__(**kwargs)

        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.sentiment_analysis_mlp_1 = nn.Linear(embed_size, embed_size)
        self.sentiment_analysis_mlp_2 = nn.Linear(embed_size, 2)
        ##### hint: self.sentiment_analysis_mlp_3 = nn.Linear(?, ?)
        self.relu = nn.ReLU()                

    def forward(self, inputs):
        # torch.unsqueeze "adds" a superficial 1 dimension to tensor (at the specified dimension), 
        # while torch.squeeze removes all superficial 1 dimensions from tensor.
        ## embed: (bz, sentence len, emb size) ->  (bz, 1, emb size) -> (bz, emb size)
        embed = torch.mean(self.embedding(inputs), dim=1).squeeze()  
        outputs = self.sentiment_analysis_mlp_1(embed)
        outputs = self.relu(outputs)
        outputs = self.sentiment_analysis_mlp_2(self.dropout(outputs))
        ##### hint: outputs = self.sentiment_analysis_mlp_3(?)
        return outputs

In [ ]:
##########################################################################################################################
# Initialize the weights.
##########################################################################################################################
def init_weights(m):
    if type(m) in (nn.Linear, nn.Conv1d):
        nn.init.xavier_uniform_(m.weight)

In [ ]:
##########################################################################################################################
# Instance the MLP and put the Glove embedding into the model
print('-----------------------------start Instance the MLP model-----------------------------')
##########################################################################################################################
glove_embedding = d2l.TokenEmbedding("glove.6b.100d") ### load embedding will spend more time.
embeds = glove_embedding[vocab.idx_to_token]
net = Sentiment_Analysis_MLP(len(vocab), embed_size)
net.apply(init_weights)
net.embedding.weight.data.copy_(embeds)
print('-----------------------------end Instance the MLP model-----------------------------')

In [ ]:
##########################################################################################################################
# Train the model.
print('-----------------------------start training process-----------------------------')
##########################################################################################################################
trainer = torch.optim.Adam(net.parameters(), lr=learning_rate)
loss = nn.CrossEntropyLoss()
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, train_epochs, devices)
print('-----------------------------end training process-----------------------------')

In [ ]:
##########################################################################################################################
# Test the model.
print('-----------------------------start testing process-----------------------------')
##########################################################################################################################

sentence_tensor = torch.tensor(vocab[sentence.split()], device=d2l.try_gpu())

label = torch.argmax(
        net(sentence_tensor.reshape(1, -1))
    )
if label == 1:
    print('The result is positive!')
else:
    print('The result is negative!')